# Twitter PainBlock

## Instalaciones y Librerias 

In [ ]:
#!pip install tweepy

In [2]:
import json
from pandas.io.json import json_normalize
import pandas as pd
import requests
import tweepy
import csv
import mysql.connector as conn

## Stream Tweet

In [3]:
# 4 authentication chains

consumer_key='jQTzrkE7vlZbg2ntJu4LESCZs'
consumer_secret='AS4B8YLOWXMcrHjJyZ8stWcm9Cp2qh0rCdIjiWaPBaTTc22tnO'
access_key='902474996-b1ltSFx5Y2EdJfi2s63pghsULdjLTF1lkW6oHBvj'
access_secret='TrZr5nGmi2Q4RcAJJ3UHbwuWkAqjcNXOTfCHLm1eOzpnn'

# authorize twitter, initialize tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

## Creating SQL Database

crea_db=conn.connect(host='localhost', user='root', passwd='123456')
cursor=crea_db.cursor()
cursor.execute('drop database if exists proyecto_final')
cursor.execute('create database proyecto_final')

In [4]:
db=conn.connect(host='localhost', user='root', passwd='123456', database='proyecto_final')

cursor=db.cursor()

cursor.execute('drop table if exists pabloiglesias_tweets')

tabla= '''
        create table pabloiglesias_tweets(
            
           tweet_id bigint ,      
           date varchar(500),
           tweet_text varchar(500),
           source varchar(500),
           in_reply_to_screen_name varchar(500),
           user_name varchar(500),
           user_screen_name varchar(500),
           location varchar(500),
           user_url varchar(500),
           n_followers int,
           verified bool 
            
        );
'''

cursor.execute (tabla)

## Tweet Streaming 

In [5]:
class TweetsListener(tweepy.StreamListener):
    
    
    def __init__(self):
        super(TweetsListener, self).__init__()


    def on_connect(self):
        print("Estoy conectado!")

    def on_status(self, status): # aquí es donde tiene que ir la base de datos si queremos guardarlo
        
        #print (json.dumps(status._json,indent=2)) 
        
        
        created_at = [] #esta
        id_tweet = [] #esta
        full_text = [] # text
        source = [] #esta
        in_reply_to_screen_name = [] #esta
        user_name = [] #esta
        user_screen_name = [] #esta
        user_location = [] #esta
        user_url = [] #esta
        user_followers_count = [] #esta
        user_verified = [] #esta
        user = [] # single-use
        
        # List Append
        
        date = str(status.created_at)
        tweet_id = int(status.id_str)
        tweet_text = status.text 
        source = status.source
        in_reply_to_screen_name = status.in_reply_to_screen_name if status.in_reply_to_screen_name else 'opinion'
        user.append(status.user)
        
        for i in user:
        
            name = i.name
            screen_name = i.screen_name
            location = i.location if i.location else ''
            url = 'https://twitter.com/' + i.screen_name
            followers_count = i.followers_count
            verified = i.verified
        
        insert_query="insert into {} ({}) values {};"\
                    .format('pabloiglesias_tweets', ','.join(['tweet_id','date',
           'tweet_text','source','in_reply_to_screen_name','user_name',
           'user_screen_name','location','user_url','n_followers','verified']), tuple([tweet_id,date,tweet_text,
                                                                                      source, in_reply_to_screen_name,name,
                                                                                      screen_name,location,url,followers_count,verified]))
        #print (insert_query)
        
        cursor.execute(insert_query)
    
        db.commit()
        
        print('Tweet Guardado')
        

    def on_error(self, status_code):
        print("Error", status_code)

stream = TweetsListener()
streamingApi = tweepy.Stream(auth=api.auth, listener=stream)
streamingApi.filter(track=["@PabloIglesias"])

In [ ]:
while 1:
    
    try:
        stream = TweetsListener()
        streamingApi = tweepy.Stream(auth=api.auth, listener=stream)
        streamingApi.filter(track= ["@PabloIglesias"])

    except:
        print ('Error')

cursor.execute('SELECT * FROM pabloiglesias_tweets').fetchall()